In [1]:
%load_ext autoreload
%autoreload 2


import torch

import torch.nn as nn
from torch.distributions import Categorical, Poisson, MixtureSameFamily
# Cd to code
import os
import sys
os.chdir('/cluster/home/kheuto01/code/prob_diff_topk')
sys.path.append('/cluster/home/kheuto01/code/prob_diff_topk')

from datasets import example_datasets, to_numpy
from torch_perturb.torch_pert_topk import PerturbedTopK
from torch_models import MixtureOfPoissonsModel, torch_bpr_uncurried

2024-07-12 13:07:54.111212: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-12 13:07:54.154828: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 13:07:54.154861: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 13:07:54.155898: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-12 13:07:54.163393: I tensorflow/core/platform/cpu_feature_guar

In [2]:
seed=360
# tracts/distributions
S=12
# history/features
H = 3
# total timepoints
T= 500
num_components=4
K=4

In [3]:
train_dataset, val_dataset, test_dataset = example_datasets(H, T, seed=seed)
train_X_THS, train_y_TS = to_numpy(train_dataset)

2024-07-12 13:08:27.748410: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-12 13:08:27.748446: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: s1cmp008.pax.tufts.edu
2024-07-12 13:08:27.748452: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: s1cmp008.pax.tufts.edu
2024-07-12 13:08:27.748487: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.129.3
2024-07-12 13:08:27.748511: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.129.3
2024-07-12 13:08:27.748517: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.129.3


In [4]:
ideal_log_rates = torch.log(torch.tensor([0+1e-8, 7, 10, 100]))
ideal_mix_weights = torch.log(1e-13 + torch.tensor(
                                [[0,1,0,0],
                                 [0,1,0,0],
                                 [0,1,0,0],
                                 [0,1,0,0],
                                 [0.3,0,0.7,0],
                                 [0.3,0,0.7,0],
                                 [0.3,0,0.7,0],
                                 [0.3,0,0.7,0],
                                 [0.9,0,0,0.1],
                                 [0.9,0,0,0.1],
                                 [0.9,0,0,0.1],
                                 [0.9,0,0,0.1]]))

In [44]:
# Instantiate the model
model = MixtureOfPoissonsModel()
step_size = 0.05
optimizer = torch.optim.Adam(model.parameters(), lr=step_size)
model.update_params(torch.cat([ideal_log_rates, ideal_mix_weights.view(-1)]))
#model.update_params(torch.cat([bpr_log_rates, bpr_mix_weights.view(-1)]))


In [45]:
M_score_func =  200
M_action = 200
train_T = train_y_TS.shape[0]
perturbed_top_K_func = PerturbedTopK(k=4)


In [46]:
mix_model = model()
sample = mix_model.sample()

In [47]:
sample

tensor([ 7.,  9.,  4.,  9., 12.,  0., 13., 15.,  0.,  0.,  0.,  0.])

In [48]:
mix_model.log_prob(sample)

tensor([-1.9038, -2.2886, -2.3944, -2.2886, -2.7129, -1.2039, -2.9752, -3.7172,
        -0.1054, -0.1054, -0.1054, -0.1054], grad_fn=<LogsumexpBackward0>)

In [49]:
losses = []
bprs = []
nlls = []

In [50]:

for epoch in range(1000):
    mix_model = model()
    
    y_sample_TMS = mix_model.sample((train_T, M_score_func))
    y_sample_action_TMS = mix_model.sample((train_T, M_action))

    ratio_rating_TMS = y_sample_action_TMS/y_sample_action_TMS.sum(dim=-1, keepdim=True)
    ratio_rating_TS =  ratio_rating_TMS.mean(dim=1)
    ratio_rating_TS.requires_grad_(True)

    #pred_y_TS = torch.mean(y_sample_action_TMS, dim=1)
    #pred_y_TS.requires_grad_(True)

    def get_log_probs_baked(param):
        distribution = model.build_from_single_tensor(param)
        log_probs_TMS = distribution.log_prob(y_sample_TMS)

        return log_probs_TMS

    jac_TMSP = torch.autograd.functional.jacobian(get_log_probs_baked, (model.params_to_single_tensor()), strategy='forward-mode', vectorize=True)

    score_func_estimator_TMSP = jac_TMSP * ratio_rating_TMS.unsqueeze(-1)
    score_func_estimator_TSP = score_func_estimator_TMSP.mean(dim=1)    

    # get gradient of negative bpr_t  with respect to ratio rating_TS
    positive_bpr_T = torch_bpr_uncurried(ratio_rating_TS, torch.tensor(train_y_TS), K=4, perturbed_top_K_func=perturbed_top_K_func)
    negative_bpr = torch.mean(-positive_bpr_T)
    
    nll = torch.sum(-mix_model.log_prob( torch.tensor(train_y_TS)))

    print(f'Neg bpr: {negative_bpr}')
    print(f'nll: {nll}')

    loss = 500*negative_bpr + nll
    print(f'Loss: {loss}')
    losses.append(loss)
    bprs.append(negative_bpr)
    nlls.append(nll)
    
    loss.backward()

    loss_grad_TS = ratio_rating_TS.grad

    gradient_TSP = score_func_estimator_TSP * torch.unsqueeze(loss_grad_TS, -1)
    gradient_P = torch.sum(gradient_TSP, dim=[0,1])

    gradient_tuple = model.single_tensor_to_params(gradient_P)

    for param, gradient in zip(model.parameters(), gradient_tuple):
        param.grad = gradient
    optimizer.step()
        
    #model.update_params(model.params_to_single_tensor() - step_size * gradient_P)


    
    


Neg bpr: -0.5340473651885986
nll: 5465.9228515625
Loss: 5198.8994140625
Neg bpr: -0.5160810351371765
nll: 5502.291015625
Loss: 5244.25048828125
Neg bpr: -0.5138471722602844
nll: 5583.00732421875
Loss: 5326.083984375
Neg bpr: -0.5174436569213867
nll: 5579.869140625
Loss: 5321.1474609375
Neg bpr: -0.5164773464202881
nll: 5557.49658203125
Loss: 5299.2578125
Neg bpr: -0.5135536789894104
nll: 5550.232421875
Loss: 5293.45556640625
Neg bpr: -0.5140158534049988
nll: 5566.26123046875
Loss: 5309.25341796875
Neg bpr: -0.5116745233535767
nll: 5549.02001953125
Loss: 5293.1826171875
Neg bpr: -0.5121027231216431
nll: 5531.09521484375
Loss: 5275.0439453125
Neg bpr: -0.5115639567375183
nll: 5516.201171875
Loss: 5260.4189453125
Neg bpr: -0.5182583928108215
nll: 5501.34033203125
Loss: 5242.2109375
Neg bpr: -0.5081961154937744
nll: 5499.42626953125
Loss: 5245.328125
Neg bpr: -0.5147411823272705
nll: 5506.16357421875
Loss: 5248.79296875
Neg bpr: -0.5120266079902649
nll: 5517.39501953125
Loss: 5261.38183593

In [48]:
log_poisson_rates, mixture_probs = model.single_tensor_to_params(model.params_to_single_tensor())
poisson_rates = torch.exp(log_poisson_rates)
mixture_probs_normalized = torch.nn.functional.softmax(mixture_probs, dim=1)

In [49]:
poisson_rates

tensor([6.7100e-01, 1.9218e+03, 4.9587e+01, 1.2280e+00],
       grad_fn=<ExpBackward0>)

In [50]:
mixture_probs_normalized

tensor([[0.0968, 0.6893, 0.1125, 0.1014],
        [0.0973, 0.5291, 0.2687, 0.1050],
        [0.0774, 0.6380, 0.1631, 0.1215],
        [0.0799, 0.6271, 0.1755, 0.1176],
        [0.1006, 0.5921, 0.1887, 0.1186],
        [0.1680, 0.1963, 0.4311, 0.2046],
        [0.1232, 0.2403, 0.4629, 0.1736],
        [0.2049, 0.2543, 0.3067, 0.2342],
        [0.2719, 0.0658, 0.1216, 0.5408],
        [0.7167, 0.0633, 0.0852, 0.1348],
        [0.5129, 0.0530, 0.0773, 0.3568],
        [0.2567, 0.0683, 0.0814, 0.5936]], grad_fn=<SoftmaxBackward0>)

In [31]:
mix_model.sample()

tensor([857., 821., 913.,   2.,   1.,   1., 875., 861.,   0.,   0.,   1.,   1.])

In [22]:
gradient_tuple

(tensor([nan, nan, nan, nan], grad_fn=<SliceBackward0>),
 tensor([[nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan]], grad_fn=<ViewBackward0>))